# Working with text data

<br>
<img src="../images/three-main-stages-of-coding-an-llm-stage1-step1.png" width="800px">
<br>

# 2.1 Understanding word embeddings

<b>Why do we need embeddings?</b>
- <span style="color:red">Deep neural network (NN) models, including LLMs cannot process text data directly. Since, the text data is categorical, it's not compatible with mathematical operations used to train NNs.</span>
- So, we need a way <span style="color:#4ea9fb">to represent non numeric data (words/text) in a continuous numbers, a format that NNs can understand and process</span>.

<b>What's an embedding?</b>
- The concept of <span style="color:#4ea9fb"><b>converting text (or other data) into numerical vector representations.</b></span>
- In other words, embedding is a mapping from discrete objects (words, image, or entire documents) into a point in coninuous space. 

<b>Different type of embeddings</b>
- While *word embeddings* are the most common form of tet embedding, there are other type of embeddings such as subword/token, sentence, paragraph, document, etc.
  - Since GPT-like LLMs learn to generate one word at a time, we will focus on **word embeddings**.
- Refer [https://prasanth.io/Knowledge/Tech/Embeddings](https://prasanth.io/Knowledge/Tech/Embeddings) for different type of embeddings.
- For *retrieval-augmentated generation*, sentence or paragraph embeddings are more popular choices.

<b>How to embed different data types?</b>
- Using a specific NN layer or another pretrained NN model, we can embed different data types - such as text, image, video, etc. 
<p style="color:black; background-color:#F5C780; padding:15px">💡Different data types require different embedding models. <span style="color:red">Embedding model designed for text data would not be suitable for embedding audio or video data.</span></p>

<br>
<img src="../images/different-embedding-models-for-different-data-types.png" width="800px">
<br>

<b><i>Word2Vec</i> - Most popular word embedding</b>
- <span style="color:#4ea9fb">The main idea behind Word2Vec is that <b>words that appear in similar contexts tend to have similar meanings</b></span>. Consequently, when projected into two-dimensional word embeddings for visualization purposes, similar terms are clustered together.
- For more details, refer [https://prasanth.io/Knowledge/Tech/Word2Vec](https://prasanth.io/Knowledge/Tech/Word2Vec)

<b>Why don't we use <i>Word2Vec</i> for LLMs?</b>
- <span style="color:#4ea9fb">LLMs commonly produce their own embeddings as part of the input layer, and are updated during training</span>.
- <span style="color:green">The advantage of optimizing the embeddings as part of the LLM training is that the embeddings are optimized to the specific data and task at hand</span>.
  - LLMs can also create contextualized output embeddings.

<b>What's an optimal Embedding Size</b>?
- It's <span style="color:#4ea9fb">a trade off between performance and effficiency</span>.

qwqww